###  Some useful imports

In [1]:
import os
import io
import datetime 
import requests
import pandas as pd
import simplejson as json
import pydgraph

###   Generate json file with test data

> Here you want to generate a json file containing data you will be able to use to call your service  
> **You can skip / remove this step if you already have a json file with test data**  
> To use your file, simply copy it to **testdata folder** eg. testdata/european.json  
> **Data should not require any further procesing before calling the service**  

In [3]:
client_stub = pydgraph.DgraphClientStub('127.0.0.1:9080')
client = pydgraph.DgraphClient(client_stub)
txn = client.txn()

# Run query.
query = """query price_request($und: string) {
  spot(func: eq(type, "stock_quote")) {
    uid
    close
  }
  
  contracts(func: eq(type, "european")) {
    uid
    strike
    expiry
    putcall
    undticker
  }
}"""

try: 
    res = client.txn(read_only=True).query(query, variables={'$und': 'AAPL'})
    json_res = json.loads(res.json)
    
    spot = json_res['spot'][0]
    contracts = json_res['contracts']
    req = [{'pricingdate': 1564669224, 'contract': contract, 'spot': spot, 'vol': 0.1, 'rate': 0.01} 
           for contract in contracts]

    with io.open(os.path.join(os.getcwd(),"../testdata/gobs.json"), 'w', encoding='utf-8') as f:
      f.write(json.dumps(req, ensure_ascii=False, indent=4))

except Exception as e:
    print(e)
    

### Call service with data from a json file

In [3]:
# load test data from json file and call service endpoint

p_requests = json.load(open(os.path.join(os.getcwd(),"../testdata/gobs.json")))
res = [requests.post('http://localhost:3000/compute', json=p_request) for p_request in p_requests]

print res

[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response

## Generate loader script